## Installed Required Packages

In [28]:
import sys
!{sys.executable} -m pip install fake-useragent

  Running setup.py bdist_wheel for fake-useragent ... done
  Stored in directory: /Users/jieyuwang/Library/Caches/pip/wheels/07/04/1d/bbd8ba7d692add504b44552504b7df239bddf56aa3387cee2b
Successfully built fake-useragent


In [235]:
!{sys.executable} -m pip install BeautifulSoup

    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/folders/pd/lj_g7cpj6nq6jqn0_47v2fxc0000gn/T/pip-build-6a69sbyc/BeautifulSoup/setup.py", line 22
        print "Unit tests have failed!"
                                      ^
    SyntaxError: Missing parentheses in call to 'print'. Did you mean print(int "Unit tests have failed!")?
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /private/var/folders/pd/lj_g7cpj6nq6jqn0_47v2fxc0000gn/T/pip-build-6a69sbyc/BeautifulSoup/


In [ ]:
!{sys.executable} -m pip install selenium

Download Chrome WebDriver: https://sites.google.com/a/chromium.org/chromedriver/getting-started

Then include the ChromeDriver location in your PATH environment variable

## Web Crapper Functions

In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re
def craw_one_page(url):
    user_agent = UserAgent()
    page = requests.get(url,headers={'user-agent':user_agent.chrome})
    content=page.content.decode('utf-8')
    soup = BeautifulSoup(content,'lxml')
    
    dict_result={}
    
    source=soup.find_all('div',attrs={'pane-header'})
    tabs = [div.string for div in source]
    tabs=tabs[:-2]
    
    
    source=soup.find_all('div',attrs={'text-content'})
    tabs_values = [div.string for div in source]
    
    for i in range(len(tabs)):
        dict_result[tabs[i]]=tabs_values[i]
    
    source=soup.find_all('a',attrs={'i'})
    keys = [div["title"] for div in source]
    
    source=soup.find_all('div',attrs={'s_val'})
    values = [div["title"] for div in source]
    
    for i in range(len(keys)):
        dict_result[keys[i]]=values[i]
    return dict_result

In [51]:
def get_number_of_search_result(source):
    browser=webdriver.Chrome()
    
    soup=BeautifulSoup(source,'lxml')
    source=soup.find('p', style={"font-size:14px"})
    #for div in source:
    return int (source.get_text()[36:-15])

In [33]:
import time 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
def crawl_search_source(search_url):
    browser=webdriver.Chrome()
    browser.get(search_url)
    number_search_result=get_number_of_search_result(browser.page_source)
    number_500=number_search_result//500
    number_100=(number_search_result-number_500*500)//100
    number_20=(number_search_result-number_500*500-number_100*100)//20
    for i in range(number_500):
        try:
            ele=browser.find_element_by_id("button_next500")
            try:
                ele.click()
            except:
                time.sleep(0.1)
                i=i-1
        except:
            pass
    for i in range(number_100):
        try:
            ele=browser.find_element_by_id("button_next100")
            try:
                ele.click()
            except:
                time.sleep(0.1)
                i=i-1
        except:
            pass
    for i in range(number_20):
        try:
            ele=browser.find_element_by_id("button_next20")
            try:
                ele.click()
            except:
                time.sleep(0.1)
                i=i-1
        except:
            pass
    return browser.page_source, number_search_result

In [34]:
from collections import OrderedDict
def find_links_from_source(soup, number_search_result):
    Tagged_text=soup.find_all('a',target={'_blank'}, )
    set_url=[]
    Tagged_text_unique=list(OrderedDict.fromkeys(Tagged_text))
    for x in Tagged_text_unique:
        url=x["href"]
        if (url[0:14]=="/plusd/cables/"):
            set_url.append(x["href"])
    return set_url[0:number_search_result]

In [5]:
import json
def save_obj(data):
    with open('data_crawled.json', 'w') as fp:
        json.dump(data, fp)

In [53]:
def crawl_search_result(set_url):
    url_head= 'https://search.wikileaks.org'
    final_result={}
    i=0
    print ("total number of links: {}".format(len(set_url)))
    for x in set_url:
        print (x[14:-5], "{} out of {} jobs done".format(i+1, len(set_url)))
        final_result[x[14:]]=craw_one_page(url_head+x)
        if (i//10==0):
            save_obj(final_result)
        i=i+1
    save_obj(final_result)
    return final_result

In [54]:
search_url="https://search.wikileaks.org/plusd/?qproject[]=ps&qproject[]=cg&qproject[]=cc&qproject[]=fp&qproject[]=ee&q=&qtfrom=1999-01-01&qtto=1999-12-31#result"

In [55]:
source, number_search_result=crawl_search_source(search_url)
soup = BeautifulSoup(source,'lxml')
set_url=find_links_from_source(soup, number_search_result)
result=crawl_search_result(set_url)

total number of links: 56
99CARACAS25_a 1 out of 56 jobs done
99CARACAS25_a 2 out of 56 jobs done
99CARACAS25_a 3 out of 56 jobs done
99CARACAS25_a 4 out of 56 jobs done
99CARACAS25_a 5 out of 56 jobs done
99STATE2487_a 6 out of 56 jobs done
99STATE2487_a 7 out of 56 jobs done
99STATE2487_a 8 out of 56 jobs done
99STATE2487_a 9 out of 56 jobs done
99STATE2487_a 10 out of 56 jobs done
99HAVANA75_a 11 out of 56 jobs done
99HAVANA75_a 12 out of 56 jobs done
99HAVANA75_a 13 out of 56 jobs done
99HAVANA75_a 14 out of 56 jobs done
99HAVANA75_a 15 out of 56 jobs done
99DAMASCUS239_a 16 out of 56 jobs done
99DAMASCUS239_a 17 out of 56 jobs done
99DAMASCUS239_a 18 out of 56 jobs done
99DAMASCUS239_a 19 out of 56 jobs done
99DAMASCUS239_a 20 out of 56 jobs done
99TUNIS305_a 21 out of 56 jobs done
99TUNIS305_a 22 out of 56 jobs done
99TUNIS305_a 23 out of 56 jobs done
99TUNIS305_a 24 out of 56 jobs done
99TUNIS305_a 25 out of 56 jobs done
99ISLAMABAD495_a 26 out of 56 jobs done
99ISLAMABAD495_a 2

In [57]:
result["99CARACAS25_a.html"]

{'Archive Status': ' -- Not Assigned -- (251288 documents)',
 'Canonical ID': ' 99CARACAS25_a',
 'Character Count': ' 10079',
 'Concepts': ' -- Not Assigned --',
 'Content': 'FOREIGN MINISTER \n \nCLASSIFIED BY THOMAS A. SHANNON, POLITICAL COUNSELOR.  REASON: 1.5 \n(B) AND (D). \n \n--------------------------------------------- -------- \nJOSE VICENTE RANGEL: VENEZUELA\'S NEW FOREIGN MINISTER \n--------------------------------------------- -------- \n \n1.(U)  PRESIDENT-ELECT HUGO CHAVEZ FRIAS NAMED JOSE VICENTE RANGEL \nAS HIS FOREIGN MINISTER DESIGNATE.  THOUGH A LAWYER BY EDUCATION \nAND A JOURNALIST BY PROFESSION, RANGEL HAS A LONG AND STORIED \nCAREER IN VENEZUELAN POLITICS.  AS A LAW STUDENT, HE LED A STUDENT \nREVOLT AGAINST THE PEREZ JIMENEZ DICTATORSHIP AND WAS EXILED FOR \nHIS PARTICIPATION.  HE LATER RETURNED TO VENEZUELA AND SERVED FOR \nNEARLY 30 YEARS IN THE CONGRESS.  DURING THIS PERIOD HE HELPED \nFOUND THE LEFTIST NEWSPAPER, EL CLARIN, WON THE NATIONAL PRIZE FOR \nJOUR